In [1]:
import pandas as pd

In [15]:
def sentiment_analysis(año:int): 
    '''Esta funcion devuelve una lista con la cantidad de registros de reseñas de usuarios 
    que se encuentren categorizados con un análisis de sentimiento según el año de lanzamiento.
        Ejemplo de retorno: {Negative = 182, Neutral = 120, Positive = 278}'''
    try:
        df = pd.read_csv("DataSets/df_sentiment.csv")
        negativos = int(df['cantidad_0'][df['ano']==año].iloc[0])
        neutros = int(df['cantidad_1'][df['ano']==año].iloc[0])
        positivos = int(df['cantidad_2'][df['ano']==año].iloc[0])

        return {"Negativos": negativos, "Neutros": neutros, "Positivos" : positivos}
    except Exception as e:
        return {"error", e}

In [16]:
sentiment_analysis(1000)

{'error', IndexError('single positional indexer is out-of-bounds')}